# Simulation of interaction between ligand/target

#### 1st cell: import modules
#### 2nd cell: function definitions
many are not used but kept as doc
#### 3rd cell: main 
To plot different curves on a same graph, 2 loops are used. One for obtaining each y value for a specific x. And another to illustrate the impact on a factor on the y value. (For example, x is the ligand concentration, y its relaxation value and z the concentration of the target). 

In [3]:
import numpy as np
from scipy import optimize, integrate
import pylab as p
import sys
from copy import copy
from random import gauss
import matplotlib.pyplot as plt

In [4]:
def R2(R2f,R2b,alpha,kex,dw):
   """ Computes a transverse relaxation rate in presence of an exchange between a free and bound states
        - R2f and R2b : transverse relaxation rates in absence or bound to the receptor
        - alpha: the bound fraction of ligand : could be an array
        - kex : the exchange rate (usually koff)
        -dw : the pulsation difference between the free and bound states
   """
   R2O = alpha*R2b+(1-alpha)*R2f
   Rex = dw**2*alpha*(1-alpha)**2
   Rex = Rex/kex
   print " R2O : %8.2e REX : %8.2e " % (R2O,Rex)
   return R2O+Rex



############################################################################
class equi():
   
   def __init__(self):
      self.diffusion = 1E8     # Diffusion limited on rate in s-1.M-1
      self.AR0 = np.array([])  # Array of receptor concentrations
      self.AL0 = np.array([])  # Array of ligand concentrations
      self.AC0 = np.array([])  # Array of inhibitor concentration
      self.YEXP = np.array([]) # Array of experimental values
      self.YCAL = np.array([]) # Array of calculated values
      self.YEXB = np.array([]) # Second array of experimental values
      self.YCAB = np.array([]) # Second array of calculated values
      self.max_time = 10     # maximal time for integration

   #-----------------------------------------------------      
   def dVR_dt(self,VR, t=0, R0=0.0, L0=0.0, C0 = 0.0, konl=0.0, kofl=0.0, konc=0.0, kofc=0.0):

      """ General differential equations for a 1 sites system 
          with 2 competiting ligands
      """
   
      # VR  : vector of different receptor species cc
       
      #R  = VR[0]
      RL = VR[0]
      RC = VR[1]
      
      R   = R0 - RL - RC      # Concentration of free receptor
      L   = L0 - RL           # Concentration of free ligand
      C   = C0 - RC           # Concentration of free competitor
      
      #dR  = kofl*RL  + kofc*RC - konl*R*L - konc*R*C
      dRL = konl*R*L - kofl*RL 
      dRC = konc*R*C - kofc*RC 
      
      return np.array([dRL, dRC])

   #-----------------------------------------------------      
   def dVR_dt_2(self,VR, t=0, R0=0.0, L0=0.0, S0=0.0, konr=0.0, kofr=0.0, kons=0.0, kofs=0.0):

      """ General differential equations for a N sites system 
          R (main receptor) and S (secondary receptor) with 1 ligands
      """
   
      # VR  : vector of different receptor species cc
       
      RL = VR[0]
      SL = VR[1]
      
      R   = R0 - RL               # Concentration of free receptor
      S   = S0 - SL               # Concentration of free secondary site
      L   = L0 - RL - SL          # Concentration of free ligand
      
      dRL = konr*R*L - kofr*RL 
      dSL = kons*S*L - kofs*SL 
      
      return np.array([dRL, dSL])
      
   #-----------------------------------------------------
   def calc_bound1s1l(self,kd):
      """ Calculate bound fractions for a one site system
          With one ligand
          kd is the dissociation constant
      """
      
      y = self.AL0/self.AR0
      b = 1+y+(kd/self.AR0)
      z = np.sqrt(b**2 - 4*y)
      X = (b - z)/2
   
      return X
            
   #-----------------------------------------------------
   def calc_bound1s2l(self,VK):
      """ Calculate bound fractions for a system with 2 competitor 
      """
      
      i = 0
      # Criteria to consider that equilibrium has been reached
      # [X(N)-X(N-1)]/R0 < tol
      
      tol = 0.0001
      mr0 = np.max(self.AR0)
      
      # Inputs
      # VK  : Array of exchange rates
      # AR0 : Vector of total concentrations of receptor  (global)
      # AL0 : Vector of total concentration of ligand     (global)
      # AC0 : Vector of total concentration of competitor (global)
           
      # Outputs
      X1 = []   # concentration of R bound to ligand 
      X2 = []   # concentration of R bound to competitor 
      #X3 = []   # concentration of R bound to competitor
      # Return -999.999 if the integration has not converged 

      konl = VK[0]
      kofl = VK[1]
      konc = VK[2]
      kofc = VK[3]
      
      # Initial values for integration
      VR0 = np.array([ 0.0, 0.0])
      
      for L0 in self.AL0:
         R0 = self.AR0[i]
         C0 = self.AC0[i]
         #print "TEST", R0, L0, C0
         #print "Test2", konl,kofl,konc,kofc
         i+=1
         t = np.linspace(0,self.max_time,10)
         VRI = integrate.odeint(self.dVR_dt, VR0, t, args=(R0,L0,C0,konl,kofl,konc,kofc))
         #print VRI.T
         
         # Computes fractional variation of concentrations
         vx1 = (VRI.T[0][9]-VRI.T[0][8])/mr0
         vx2 = (VRI.T[1][9]-VRI.T[1][8])/mr0
         #vx3 = (VRI.T[2][9]-VRI.T[2][8])/mr0
         
         #if (vx1<tol and vx2<tol and vx3<tol) :
         if (vx1<tol and vx2<tol) :   
            X1.append(VRI.T[0][9])
            X2.append(VRI.T[1][9])
            #X3.append(VRI.T[2][9])
         else :
            X1.append(np.NaN)
            X2.append(np.NaN)
            #X3.append(np.NaN)
            
      #return(X1,X2,X3)
      return(X1,X2)
    
    #-----------------------------------------------------
    
   #-----------------------------------------------------
   def calc_bound_Ns(self,VK):
      """ Calculate bound fractions for a system with N sites
          with 1 of higher affinity than the other
      """
      
      i = 0
      # Criteria to consider that equilibrium has been reached
      # [X(N)-X(N-1)]/R0 < tol
      
      tol = 0.0001
      mr0 = np.max(self.AR0)
      
      # Inputs
      # VK  : Array of exchange rates
      # AR0 : Vector of total concentrations of receptor  (global)
      # AL0 : Vector of total concentration of ligand     (global)
           
      # Outputs
      X1 = []   # concentration of Ligand bound to main
      X2 = []   # concentration of Ligand bound to secondary
      # Return -999.999 if the integration has not converged 

      konr = VK[0]
      kofr = VK[1]
      kons = VK[2]
      kofs = VK[3]
      
      # Initial values for integration
      VR0 = np.array([ 0.0, 0.0])
      
      for L0 in self.AL0:
         R0 = self.AR0[i]
         S0 = self.AC0[i]

         i+=1
         t = np.linspace(0,self.max_time,10)
         VRI = integrate.odeint(self.dVR_dt, VR0, t, args=(R0,S0,L0,konr,kofr,kons,kofs))
         
         # Computes fractional variation of concentrations
         vx1 = (VRI.T[0][9]-VRI.T[0][8])/mr0
         vx2 = (VRI.T[1][9]-VRI.T[1][8])/mr0
         #vx3 = (VRI.T[2][9]-VRI.T[2][8])/mr0
         
         #if (vx1<tol and vx2<tol and vx3<tol) :
         if (vx1<tol and vx2<tol) :   
            X1.append(VRI.T[0][9])
            X2.append(VRI.T[1][9])
            #X3.append(VRI.T[2][9])
         else :
            X1.append(np.NaN)
            X2.append(np.NaN)
            #X3.append(np.NaN)
            
      #return(X1,X2,X3)
      return(X1,X2)
    
    #-----------------------------------------------------

   #-----------------------------------------------------
   def write_1d(self,fname):
      """ Write an experiment as a text file with 3 column tabulated format 
          R0 L0 Y
      """
      file = open(fname,'w')
      i = 0
      for R0 in self.AR0 :
         L0 = self.AL0[i]
         Y = self.YCAL[i]
         file.write('%8.3e %8.3e %8.4f \n' % (R0, L0, Y))
         i += 1
      
      print " %3d values written in file %s \n " % (i,fname)
      return True

   #-----------------------------------------------------
   def model1(self,PAR,CNST=[1E-9]) :
      """ model 1 : 2 parameters fit of low affinity site using 
          ppmax and kd2 as parameters
          CNST contains the high affinity kd value (kd1)
      """
      # Initialisation of kinetic constants
      kon1 = self.diffusion # Diffusion limited on rate in s-1.M-1
      kon2 = kon1
      kd1 = CNST[0]
      kd2 = PAR[0]
      kof1 = kd1*kon1
      kof2 = kd2*kon2 
      VK = np.array([kon1, kof1, kon2, kof2])
       
      ppmax = PAR[1]
      X1,X2,X3 = self.calc_bound2s(VK)
      X4=(np.array(X2)+np.array(X3))/self.AR0  # X4 fraction of occupied low aff site
      
      self.YCAL=X4*ppmax
      
      dist = np.subtract(self.YEXP,self.YCAL)
      
      return dist
      
   #-----------------------------------------------------
   def model2(self,PAR,CNST=[1E-3]) :
      """ model 2 : 2 parameters fit for STD experiments
          parameters : high affinity Kd and STDmax
          CNST contains the low affinity kd value (kd2)
      """
      # Initialisation of kinetic constants
      kon1 = self.diffusion # Diffusion limited on rate in s-1.M-1
      kon2 = kon1
      kd1 = CNST[0]
      kd2 = PAR[0]
      kof1 = kd1*kon1
      kof2 = kd2*kon2 
      VK = np.array([kon1, kof1, kon2, kof2])
      
      stdmax = PAR[1]
      X1,X2,X3 = self.calc_bound2s(VK)
      
      # Fraction of bound ligand 
      X4=(np.array(X1)+np.array(X2)+2*np.array(X3))/self.AR0
           
      self.YCAL=X4*stdmax
      
      dist = np.subtract(self.YEXP,self.YCAL)
      
      return dist

   #-----------------------------------------------------
   def model0(self,PAR,CNST=[]) :
      """ model 1 : 2 parameters fit of high affinity site using 
          ppmax and kd as parameters
      """
      
      kd = PAR[0]
      ppmax = PAR[1]
      
      self.YCAL = ppmax*self.calc_bound1s(kd)
      
      dist = np.subtract(self.YEXP,self.YCAL)
      
      return dist

   #-----------------------------------------------------
   def model3(self,PAR,CNST=[0.0, 0.0]) :
      """ model 3 : 4 parameters fit of a 2D correlation peak trajectory 
          ppmx1, ppmy1 ; ppmx2, ppmy2 as parameters (chemical shifts of full occupied sites)
          CNST contains the high and low affinity kd values
          Assume that the first point is at 0.0 concentration of ligand
      """
      # Initialisation of kinetic constants
      kon1 = self.diffusion # Diffusion limited on rate in s-1.M-1
      kon2 = kon1
      kd1 = CNST[0]
      kd2 = CNST[1]
      kof1 = kd1*kon1
      kof2 = kd2*kon2 
      VK = np.array([kon1, kof1, kon2, kof2])
       
      v1 = np.array([PAR[0],PAR[1]])
      v2 = np.array([PAR[2],PAR[3]])
      # Assume that the first point is the origin of the titration
      vo = np.array([self.YEXP[0],self.YEXB[0]])
      
      # Computes sites occupencies
      X1,X2,X3 = self.calc_bound2s(VK)
      X4=(np.array(X2)+np.array(X3))/self.AR0  # X4 fraction of occupied low aff site
      X5=(np.array(X1)+np.array(X3))/self.AR0  # X5 fraction of occupied high aff site
      
      i=0
      ycal=[]
      ycab=[]
      for x1 in X5:
         x2 = X4[i]
         vi = vo + x1*(v1-vo) + x2*(v2-vo)
         ycal.append(vi[0])
         ycab.append(vi[1])
         i+=1

      self.YCAL = np.array(ycal)
      self.YCAB = np.array(ycab)       
      dist = np.subtract(self.YEXP,self.YCAL)**2 + np.subtract(self.YEXB,self.YCAB)**2
      
      return dist
      
   #-----------------------------------------------------
   def fit(self,fun,PAR,CNST,nrepeat=1) :
      """ Fit the target function 
          PAR is the vector of parameters to be adjusted
          CNST is a vector of constant values
      """
      
      PAROPT,n = optimize.leastsq(fun,PAR,args=(CNST))
      PARSTD = []
      
      if (nrepeat > 1) :
      
      # Evaluate the rms between calculated and experimental points
         dist = np.subtract(self.YEXP,self.YCAL)
         khi = np.linalg.norm(dist)
         Y = self.YCAL.copy() # Store result from first fit
         NP=len(Y)
         noise = khi/np.sqrt(NP)
         apar = []
         for i in range(1,nrepeat+1) :
            self.YCAL = Y.copy()
            self.noise(nlvl=noise)
            parmc,n = optimize.leastsq(fun,PAR,args=(CNST))
            apar.append(parmc)
            
         apar = np.array(apar)
         
         PARSTD = apar.T.std(1)
         
         return PAROPT,PARSTD
         
      else : return PAROPT

   #-----------------------------------------------------
   def simul(self,fun,PAR,CNST=[],NP=100,R0=1E-4,AL0B=[0.0, 1E-3]) :
      """ Fill AR0 and AL0 vectors for an virtual titration experiment
          fun is contains the model used to calculate the concentrations
          PAR is the vector of parameters which depends on the model
          CNST is a vector of constant values which depends on the model
          NP Number of titration points
          R0 Receptor concentration (which is kept constant)
          AL0B Boundaries of ligand concentrations
      """
      
      self.AR0 = np.ones(NP)*R0
      self.AL0 = np.arange(AL0B[0],AL0B[1],(AL0B[1]-AL0B[0])/NP)
      
      yexp = self.YEXP.copy()
      yexb = self.YEXB.copy()
      nexp = len(self.YEXP)
      self.YEXP = np.zeros(NP) # and fill YEXP with dummy values
      self.YEXB = np.zeros(NP) # and fill YEXP with dummy values

      for i in range(0,nexp) :
         self.YEXP[i] = yexp[i]
         self.YEXB[i] = yexb[i]

      if (fun == "model0") : 
         d = self.model0(PAR,CNST)
      elif (fun == "model1") :
         d = self.model1(PAR,CNST)
      elif (fun == "model2") :
         d = self.model2(PAR,CNST)      
      elif (fun == "model3") :
         d = self.model3(PAR,CNST)      
      

   #-----------------------------------------------------   
   def plot(self,fig=1, xaxis="L0", yaxis="YEXP", style='or', draw=True):
      """ Plot the content of an experiment
          fig is the figure number
          xaxis is either L0, R0 or YEXP
          yaxis is either L0, R0, YEXP or YCAL
          style is defines the plot style
          draw toggles the drawing (use draw=False to hold the plot)
      """
      if (xaxis == "L0"): 
         X = self.AL0
      elif (xaxis == "R0"):
         X = self.AR0     
      elif (xaxis == "YEXP"):
         X = self.YEXP
      elif (xaxis == "YEXB"):
         X = self.YEXB     
      elif (xaxis == "YCAL"):
         X = self.YCAL
      elif (xaxis == "YCAB"):
         X = self.YCAB 
     

      if (yaxis == "L0"): 
         Y = self.AL0
      elif (yaxis == "R0"):
         Y = self.AR0     
      elif (yaxis == "YEXP"):
         Y = self.YEXP
      elif (yaxis == "YEXB"):
         Y = self.YEXB 
      elif (yaxis == "YCAL"):
         Y = self.YCAL
      elif (yaxis == "YCAB"):
         Y = self.YCAB


      f = p.figure(fig)
      p.plot(X,Y,style)
      if draw: p.show()     

   #-----------------------------------------------------   
   def noise(self,nlvl=0):
      """ Add a constant level of noise to the YCAL vector
          and copy the result to YEXP
          affected data: YEXP
      """

      YV=[]
    
      for y in self.YCAL:
         yn=y+gauss(0,nlvl)
         YV.append(yn)
         
      self.YEXP = np.array(YV)
         
         
############################################################################


In [ ]:
def main():
   fig = plt.figure()
   ax = fig.add_subplot(1,1,1)
   a = equi()  

   ######## Fixed constants ###################

   R2f = 3                                    # R2 free state relaxation rate 
   R2r = 1000                                 # R2 ribosome bound state relaxation rate
   R2s = 10 



   N = 1                                     # Number of binding sites
   I0 = 1000.                                 # Max intensity
   dw = 100.                                    # rad.s-1

   concR = [0.9E-12, 0.5E-6, 1.15E-6]               # for receptor concentration variation
   #kdl = [1E-6, 10E-6, 50E-6, 100E-6, 200E-6]     # for ligand Kd variation
   #concC = [0, 50E-6, 1E-3, 100]                  # for competitor concentration variation

   ########  Variables ######################## 
    
   for concr in concR:                             # "z" value
       Kdr = 1E-6                           # Molar -- Kd ligand
       Kds = 1E-3                             # Molar -- Kd competitor
       
       konl  = 1E8                            # ligand "on" rate in M-1s-1 
       kofl = Kdl*konl
       konc  = 1E8                         # Competitor "on" rate in M-1s-1
       kofc = Kdc*konc
       Vector = np.array([ konl, kofl, konc, kofc])

       RLf = 0
       RCf = 0
       fracLb = 0
       boundL = []
       boundC = []
       I = []
       Rl = []


       ########### With the ligand concentration in abscisse #########
    
       #Ligand = [10*1E-6, 25*1E-6, 50*1E-6, 75*1E-6, 100*1E-6, 200*1E-6]
       #Ligand = np.linspace(10E-6, 0.001, num=100)
       #Compet = np.logspace(1E-7, 0.001, num=100)  
       #Compet = np.linspace(1E-7, 0.001, num=100)
       Ligand = np.logspace(-5, -3, num=100)


       for concl in Ligand:                           # "x" value
            a.AR0 = np.array([concr])
            a.AL0 = np.array([concl])
            a.AC0 = np.array([0])
            
            RLf, RCf = a.calc_bound1s2l(Vector)
            fracLb = N*(RLf[0]/a.AL0)
            
            boundL.append(fracLb)
            boundC.append(RCf)
            
            R2obs = R2(R2f, R2b, fracLb, kofl, dw)    # "y" value
            peak_intensity = I0*(a.AL0-RLf)/R2obs
            
            Rl.append(R2obs)
            I.append(peak_intensity)
            




       line, = ax.plot(Ligand, Rl, '-o')                # matplotlib plot colors in this order: blue, green, red, cyan...
       ax.set_xscale('log')                           # un-comment if the x-scale is not linear but logarithmic
   plt.show()




############################################################################
if __name__ == '__main__': main()


In [ ]:
np.log10(200)

In [ ]:
Ligtest = np.logspace(0, 2.3, num=7)
print Ligtest



In [104]:
## 2e version

def main():
   fig = plt.figure()
   ax = fig.add_subplot(1,1,1)
   a = equi()  

   ######## Fixed constants ###################

   R2f = 3                                    # R2 free state relaxation rate 
   R2r = 200                                  # R2 ribosome bound state relaxation rate
   R2s = 200

   N = 1                                     # Number of binding sites
   Ns = 5
   I0 = 1000.                                 # Max intensity
   dw = 10.                                    # rad.s-1

   concR = [1.15E-6]               # for receptor concentration variation
   #kdl = [1E-6, 10E-6, 50E-6, 100E-6, 200E-6]     # for ligand Kd variation
   #concC = [0, 50E-6, 1E-3, 100]                  # for competitor concentration variation

   ########  Variables ######################## 
    
   for concr in concR:                             # "z" value
       Kdr = 5E-6                           # Molar -- Kd ligand
       Kds =  800E-6                          # Molar -- Kd competitor
       
       konr  = 1E8                        # ligand "on" rate in M-1s-1 
       kofr = Kdr*konr
       #kofr = 1E0
       #konr = 1E6
       #Kdr = float(float(kofr)/float(konr))
       kons  = 1E8                          # Competitor "on" rate in M-1s-1
       kofs = Kds*kons
       Vector = np.array([ konr, kofr, kons, kofs])

       RLf = 0
       SLf = 0
       fracLbR = 0
       fracLbS = 0
       boundR = []
       boundS = []
       I = []
       Rr = []
       Rs = []
       Rt = []
      


       ########### With the ligand concentration in abscisse #########
    
       #Ligand = [10*1E-6, 25*1E-6, 50*1E-6, 75*1E-6, 100*1E-6, 200*1E-6]
       #Ligand = np.linspace(10E-6, 0.001, num=100)
       #Compet = np.logspace(1E-7, 0.001, num=100)  
       #Compet = np.linspace(1E-7, 0.001, num=100)
       Ligand = np.logspace(-5, -3, num=100)


       for concl in Ligand:                           # "x" value
            a.AR0 = np.array([1E-6])
            a.AL0 = np.array([concl])
            a.AC0 = np.array([1E-6])
            
            RLf, SLf = a.calc_bound_Ns(Vector)
            fracLbR = N*(RLf[0]/a.AL0)
            fracLbS = Ns*(SLf[0]/a.AL0)
            
            boundR.append(fracLbR)
            boundS.append(fracLbS)
            
            R2obs_r = R2(R2f, R2r, fracLbR, kofr, dw)    # "y" value
            R2obs_s = R2(R2f, R2s, fracLbS, kofs, dw)
            R2obs_tot = R2obs_r + R2obs_s
            #peak_intensity = I0*(a.AL0-RLf)/R2obs
            
            Rr.append(R2obs_r)
            Rs.append(R2obs_s)
            Rt.append(R2obs_tot)
            #I.append(peak_intensity)
            




       line, = ax.plot(Ligand, Rr, '-o', label='specific')                # matplotlib plot colors in this order: blue, green, red, cyan...
       line, = ax.plot(Ligand, Rs, '-o', label='non-spe')
       line, = ax.plot(Ligand, Rt, '-o', label='sum') 
       ax.legend()
       ax.set_xscale('log')                           # un-comment if the x-scale is not linear but logarithmic
   plt.show()




############################################################################
if __name__ == '__main__': main()



 R2O : 5.85e+00 REX : 2.81e-03 
 R2O : 4.04e+00 REX : 6.52e-06 
 R2O : 5.72e+00 REX : 2.68e-03 
 R2O : 4.04e+00 REX : 6.52e-06 
 R2O : 5.59e+00 REX : 2.56e-03 
 R2O : 4.04e+00 REX : 6.52e-06 
 R2O : 5.47e+00 REX : 2.45e-03 
 R2O : 4.04e+00 REX : 6.51e-06 
 R2O : 5.36e+00 REX : 2.34e-03 
 R2O : 4.04e+00 REX : 6.51e-06 
 R2O : 5.25e+00 REX : 2.23e-03 
 R2O : 4.04e+00 REX : 6.51e-06 
 R2O : 5.15e+00 REX : 2.13e-03 
 R2O : 4.04e+00 REX : 6.50e-06 
 R2O : 5.05e+00 REX : 2.04e-03 
 R2O : 4.03e+00 REX : 6.50e-06 
 R2O : 4.95e+00 REX : 1.95e-03 
 R2O : 4.03e+00 REX : 6.49e-06 
 R2O : 4.86e+00 REX : 1.86e-03 
 R2O : 4.03e+00 REX : 6.49e-06 
 R2O : 4.78e+00 REX : 1.77e-03 
 R2O : 4.03e+00 REX : 6.48e-06 
 R2O : 4.70e+00 REX : 1.69e-03 
 R2O : 4.03e+00 REX : 6.48e-06 
 R2O : 4.62e+00 REX : 1.62e-03 
 R2O : 4.03e+00 REX : 6.47e-06 
 R2O : 4.54e+00 REX : 1.54e-03 
 R2O : 4.03e+00 REX : 6.47e-06 
 R2O : 4.47e+00 REX : 1.47e-03 
 R2O : 4.03e+00 REX : 6.46e-06 
 R2O : 4.40e+00 REX : 1.41e-03 
 R2O : 4